In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
TEST_DIR = 'Test'
TRAIN_DIR = 'Train'

In [3]:
train_bureau_path = os.path.join(TRAIN_DIR, 'train_bureau.xlsx')
train_bureau_pd = pd.read_excel(train_bureau_path)
train_bureau_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560844 entries, 0 to 560843
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   ID                         560844 non-null  int64         
 1   SELF-INDICATOR             560844 non-null  bool          
 2   MATCH-TYPE                 560844 non-null  object        
 3   ACCT-TYPE                  560844 non-null  object        
 4   CONTRIBUTOR-TYPE           560844 non-null  object        
 5   DATE-REPORTED              557161 non-null  datetime64[ns]
 6   OWNERSHIP-IND              560844 non-null  object        
 7   ACCOUNT-STATUS             560844 non-null  object        
 8   DISBURSED-DT               528694 non-null  datetime64[ns]
 9   CLOSE-DT                   309086 non-null  object        
 10  LAST-PAYMENT-DATE          241561 non-null  datetime64[ns]
 11  CREDIT-LIMIT/SANC AMT      15159 non-null   object  

In [4]:
train_data_path = os.path.join(TRAIN_DIR, 'train_Data.xlsx')
train_data_pd = pd.read_excel(train_data_path)

In [5]:
test_bureau_path = os.path.join(TEST_DIR, 'test_bureau.xlsx')
test_bureau_pd = pd.read_excel(test_bureau_path)

In [6]:
test_data_path = os.path.join(TEST_DIR, 'test_Data.xlsx')
test_data_pd = pd.read_excel(test_data_path)

In [7]:
# corrmat = train_data_pd.corr()
# fix, ax = plt.subplots(figsize=(12,9))
# sns.heatmap(corrmat, square=True, annot=True, linewidths=0.5, cmap='Blues')

In [8]:
train_data_pd.dtypes[train_data_pd.dtypes == 'object']

Frequency        object
InstlmentMode    object
LoanStatus       object
PaymentMode      object
Area             object
SEX              object
City             object
State            object
Top-up Month     object
dtype: object

In [9]:
train_data_pd.isnull().sum()

ID                     0
Frequency              0
InstlmentMode          0
LoanStatus             0
PaymentMode            0
BranchID               0
Area               11653
Tenure                 0
AssetCost              0
AmountFinance          0
DisbursalAmount        0
EMI                    0
DisbursalDate          0
MaturityDAte           1
AuthDate               0
AssetID                0
ManufacturerID         0
SupplierID             0
LTV                    0
SEX                   59
AGE                   59
MonthlyIncome        234
City               11256
State                  0
ZiPCODE              372
Top-up Month           0
dtype: int64

In [10]:
for id in train_data_pd['BranchID'].unique():
    area = train_data_pd[train_data_pd['BranchID'] == id].Area.mode()
    print(f'Id = {id}: {area}')

Id = 1: Series([], dtype: object)
Id = 333: 0    BHOPAL
dtype: object
Id = 125: 0    GUNA
dtype: object
Id = 152: 0    BILASPUR
dtype: object
Id = 5: 0    RAIPUR
dtype: object
Id = 154: Series([], dtype: object)
Id = 79: Series([], dtype: object)
Id = 8: 0    BHUBANESHWAR
dtype: object
Id = 194: 0    CUTTACK
dtype: object
Id = 316: 0    BHUBANESHWAR
dtype: object
Id = 9: 0    BURDWAN
dtype: object
Id = 10: Series([], dtype: object)
Id = 231: Series([], dtype: object)
Id = 289: 0    DURGAPUR
dtype: object
Id = 149: 0    SILIGURI
dtype: object
Id = 229: 0    BURDWAN
dtype: object
Id = 16: 0    BHUBANESHWAR
dtype: object
Id = 17: Series([], dtype: object)
Id = 19: 0    UDAIPUR
dtype: object
Id = 20: 0    CHANDIGARH
dtype: object
Id = 200: 0    LUDHIANA   RANI JHANSI ROAD
dtype: object
Id = 21: 0    JAIPUR
dtype: object
Id = 281: 0    JAIPUR
dtype: object
Id = 110: 0    JAIPUR
dtype: object
Id = 205: 0    JAIPUR
dtype: object
Id = 341: 0    JAIPUR
dtype: object
Id = 22: 0    JALANDHAR
dtyp

In [11]:
area = train_data_pd[train_data_pd['BranchID'] == 24].Area.mode()
area = 'NONE' if area.empty else area[0]
area

'KANPUR'

In [12]:
pd.to_datetime(train_data_pd['AuthDate'])

0        2012-02-10
1        2012-03-31
2        2017-06-17
3        2013-11-29
4        2011-12-08
            ...    
128650   2018-09-22
128651   2018-09-22
128652   2018-11-23
128653   2018-12-20
128654   2018-12-31
Name: AuthDate, Length: 128655, dtype: datetime64[ns]

In [13]:
def aut_date_to_year(data):
    datetime = pd.to_datetime(data['AuthDate'])
    data['AuthYear'] = data.AuthDate.dt.year
    data.drop('AuthDate', axis=1, inplace=True)

In [14]:
def get_areas_by_ids(data):
    areas = {}
    for id in data['BranchID'].unique():
        area = data[data['BranchID'] == id].Area.mode()
        area = 'NONE' if area.empty else area[0]
        areas[id] = area
    return areas

In [15]:
def replace_areas(data):
    areas_dict = get_areas_by_ids(data)
    null_areas = data['Area'].isnull()
    data.Area = data[null_areas].Area = data['BranchID'].map(areas_dict)

In [16]:
def get_cities_by_ids(data):
    cities = {}
    for id in data['BranchID'].unique():
        city = data[data['BranchID'] == id].City.mode()
        city = 'NONE' if city.empty else city[0]
        cities[id] = city
    return cities

In [17]:
def replace_city(data):
    city_dict = get_cities_by_ids(data)
    null_city = data['City'].isnull()
    data.City = data[null_city].Area = data['BranchID'].map(city_dict)

In [18]:
def dis_date_to_year(data):
    data['DisbursalYear'] = data.DisbursalDate.dt.year
    data.drop('DisbursalDate', axis=1, inplace=True)

In [19]:
def mat_date_to_year(data):
    data['MaturityYear'] = data.MaturityDAte.dt.year
    data.drop('MaturityDAte', axis=1, inplace=True)

In [20]:
def fill_income_nan(data):
    data['MonthlyIncome'].fillna((data['MonthlyIncome'].mean()), inplace=True)

In [21]:
def fill_mat_date_nan(data):
    dif = (data['MaturityYear'] - data['DisbursalYear']).mode()[0]
    data['MaturityYear'].fillna((data['DisbursalYear'] + dif), inplace=True)

In [22]:
def drop_sex(data):
    try:
        data.drop('SEX', axis=1, inplace=True)
        return True
    except:
        return False

In [23]:
def drop_age(data):
    try:
        data.drop('AGE', axis=1, inplace=True)
        return True
    except:
        return False

In [24]:
def drop_zipcode(data):
    try:
        data.drop('ZiPCODE', axis=1, inplace=True)
        return True
    except:
        return False

In [25]:
def drop_manuf_id(data):
    try:
        data.drop('ManufacturerID', axis=1, inplace=True)
        return True
    except:
        return False

In [67]:
def add_bureau(data, bureau):
    report_counts = (bureau.groupby(['ID']).size()).to_frame('Report-Counts')
    data = data.join(report_counts, on='ID')

In [79]:
def preprocesing(data, bureau):
    # replace_city(data)
    # replace_areas(data)
    # fill_income_nan(data)
    # dis_date_to_year(data)
    # mat_date_to_year(data)
    # aut_date_to_year(data)
    # fill_mat_date_nan(data)
    # drop_age(data)
    # drop_sex(data)
    # drop_zipcode(data)
    add_bureau(data, bureau)
    # drop_manuf_id(data)

In [80]:
train_data_pd.isnull().sum()

ID                 0
Frequency          0
InstlmentMode      0
LoanStatus         0
PaymentMode        0
BranchID           0
Area               0
Tenure             0
AssetCost          0
AmountFinance      0
DisbursalAmount    0
EMI                0
AssetID            0
SupplierID         0
LTV                0
MonthlyIncome      0
City               0
State              0
Top-up Month       0
DisbursalYear      0
MaturityYear       0
AuthYear           0
dtype: int64

In [81]:
train_data_pd['ManufacturerID']

KeyError: 'ManufacturerID'

In [ ]:
preprocesing(train_data_pd, train_bureau_pd)

In [83]:
train_data_pd.isnull().sum()

ID                 0
Frequency          0
InstlmentMode      0
LoanStatus         0
PaymentMode        0
BranchID           0
Area               0
Tenure             0
AssetCost          0
AmountFinance      0
DisbursalAmount    0
EMI                0
AssetID            0
SupplierID         0
LTV                0
MonthlyIncome      0
City               0
State              0
Top-up Month       0
DisbursalYear      0
MaturityYear       0
AuthYear           0
dtype: int64

In [84]:
test_data_pd.isnull().sum()

ID                 0
Frequency          0
InstlmentMode      0
LoanStatus         0
PaymentMode        0
BranchID           0
Area               0
Tenure             0
AssetCost          0
AmountFinance      0
DisbursalAmount    0
EMI                0
AssetID            0
SupplierID         0
LTV                0
MonthlyIncome      0
City               0
State              0
DisbursalYear      0
MaturityYear       0
AuthYear           0
dtype: int64

In [85]:
preprocesing(test_data_pd, test_bureau_pd)

In [86]:
test_data_pd.isnull().sum()

ID                 0
Frequency          0
InstlmentMode      0
LoanStatus         0
PaymentMode        0
BranchID           0
Area               0
Tenure             0
AssetCost          0
AmountFinance      0
DisbursalAmount    0
EMI                0
AssetID            0
SupplierID         0
LTV                0
MonthlyIncome      0
City               0
State              0
DisbursalYear      0
MaturityYear       0
AuthYear           0
dtype: int64

In [91]:
report_counts = (train_bureau_pd.groupby(['ID']).size()).to_frame('Report-Counts')
train_data_pd = train_data_pd.join(report_counts, on='ID')
report_counts = (test_bureau_pd.groupby(['ID']).size()).to_frame('Report-Counts')
test_data_pd = test_data_pd.join(report_counts, on='ID')

In [92]:
train_data_pd.to_csv('train_data.csv', index=False)
test_data_pd.to_csv('test_data.csv', index=False)

In [94]:
train_data_pd

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,...,SupplierID,LTV,MonthlyIncome,City,State,Top-up Month,DisbursalYear,MaturityYear,AuthYear,Report-Counts
0,1,Monthly,Arrear,Closed,PDC_E,1,NONE,48,450000,275000.0,...,21946,61.11,35833.33,BHOPAL,MADHYA PRADESH,> 48 Months,2012,2016.0,2012,9
1,2,Monthly,Advance,Closed,PDC,333,BHOPAL,47,485000,350000.0,...,34802,70.00,666.67,SEHORE,MADHYA PRADESH,No Top-up Service,2012,2016.0,2012,13
2,3,Quatrly,Arrear,Active,Direct Debit,1,NONE,68,690000,519728.0,...,127335,69.77,45257.00,BHOPAL,MADHYA PRADESH,12-18 Months,2017,2023.0,2017,31
3,7,Monthly,Advance,Closed,Billed,125,GUNA,48,480000,400000.0,...,25094,80.92,20833.33,ASHOK NAGAR,MADHYA PRADESH,> 48 Months,2013,2017.0,2013,4
4,8,Monthly,Arrear,Closed,Billed,152,BILASPUR,44,619265,440000.0,...,21853,71.05,27313.67,BILASPUR,CHATTISGARH,36-48 Months,2011,2015.0,2011,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128650,143390,Half Yearly,Arrear,Closed,Direct Debit,424,PANIPAT,24,470000,265601.0,...,48879,40.17,65333.33,SONIPAT,HARYANA,24-30 Months,2018,2020.0,2018,2
128651,143391,Half Yearly,Arrear,Closed,Direct Debit,424,PANIPAT,24,460000,275630.0,...,48879,59.92,83333.33,SONIPAT,HARYANA,No Top-up Service,2018,2020.0,2018,1
128652,143393,Monthly,Arrear,Active,Direct Debit,424,PANIPAT,23,545000,300733.0,...,44118,52.38,248500.00,SONIPAT,HARYANA,No Top-up Service,2018,2020.0,2018,4
128653,143394,Half Yearly,Arrear,Active,Direct Debit,424,PANIPAT,35,350000,250962.0,...,48879,50.37,84500.00,SONIPAT,HARYANA,No Top-up Service,2018,2021.0,2018,1
